In [1]:
import sys
sys.path = ['../../src'] + sys.path # this ensures the notebook uses the source version of Tarski

In [2]:
import pyrddl

In [3]:
from tarski.io import rddl

In [4]:
mr_reader = rddl.Reader('../../tests/data/rddl/Mars_Rover.rddl')

Generating LALR tables


In [5]:
mr_reader.translate_rddl_model()

In [6]:
mr_reader.rddl_model

# RDDL Domains

The name of the domain

In [7]:
mr_reader.rddl_model.domain.name

'simple_mars_rover'

In [8]:
domain = mr_reader.rddl_model.domain
instance = mr_reader.rddl_model.instance

## Requirements

In [9]:
for req in domain.requirements:
    print(req)

concurrent
reward-deterministic
intermediate-nodes
constrained-state


As described [here](http://users.cecs.anu.edu.au/~ssanner/IPPC_2011/RDDL.pdf) these requirements mean:

 - ```concurrent``` - Multiple actions can be executed at a given time step.
 - ```reward-deterministic``` - The rewards are deterministic.
 - ```intermediate-nodes``` - The domain uses _intermediate pvariable nodes_. These are defined with the ```interm-fluent``` keyword and they're stratified. This is the device used by RDDL to ensure that the transition function is well defined, as these intermediate variables have their values defined in terms of state variables (which have a level of stratification of 0).
 - ```constrained-state``` - The domain uses _state constraints_.

## Instance Parameters

In [10]:
instance.horizon

40

In [11]:
instance.discount

1.0

In [12]:
instance.max_nondef_actions

'pos-inf'

## Types

We can access the types defined in the domain via the ```types``` attribute

In [13]:
print(domain.types)

[('picture-point', 'object')]


a list of pairs of _type names_ and its parent type (or sort in Tarski's terms). The [set of built-in types](http://users.cecs.anu.edu.au/~ssanner/IPPC_2011/RDDL.pdf) in RDDL are:

 - ```object``` - an arbitrary set of names, it has no domain. Equivalent to Tarski's built-in ```Object``` sort.
 - ```real``` - the real numbers.
 - ```integer``` - the integer numbers. Using this type is enabled by the requirements ```integer-variables```.
 - ```bool``` - Boolean value of $\top$ or $\bot$.
 
Non-built in types are specializations of ```object``` as is the case of ```picture-point``` above, and correspond directly with user-defined Tarski sorts.

## Variables

RDDL variables map into FSTRIPS terms and (non-built in) atoms. Let's take a look at them:

### Non-Fluents

Non-fluent variables correspond to static terms and atoms. In the case of the Mars Rover domain we're using in this example, these are

In [14]:
non_fluent_terms = domain.non_fluents
for k, v in non_fluent_terms.items():
    params = []
    if v.param_types is not None:
        params = v.param_types
    signature = tuple(params + [v.range])
    print('Symbol: {} Signature: {}'.format(k, signature))

Symbol: MAX_TIME/0 Signature: ('real',)
Symbol: MOVE_VARIANCE_MULT/0 Signature: ('real',)
Symbol: PICT_ERROR_ALLOW/1 Signature: ('picture-point', 'real')
Symbol: PICT_YPOS/1 Signature: ('picture-point', 'real')
Symbol: PICT_VALUE/1 Signature: ('picture-point', 'real')
Symbol: PICT_XPOS/1 Signature: ('picture-point', 'real')


Worth noting:
 - the ```param_types``` attribute is None when the domain of the term is the empty set
 
Associated to the RDDL instance we can find the specification of the domains of custom defined types and the actual values initially set to the set of non-fluent terms.

In [15]:
print(instance.non_fluents)

pics3


Rather than an object, ```pyrddl.Instance``` objects attribute ```non_fluents``` is the name of the syntactic block used to group constant declarations. The actual object is actually part of the full RDDL model specification 

In [16]:
constant_symbols_def = mr_reader.rddl_model.non_fluents

The specification of the domains of any custom types can be found on the attribute ```objects```

In [17]:
print(constant_symbols_def.objects)

[('picture-point', ['p1', 'p2', 'p3'])]


and the definitions of the constant fluents are given in the list of fluent initializers ```init_non_fluent```. These consist of pairs of variable-free terms and values.

In [18]:
print(constant_symbols_def.init_non_fluent)

[(('MAX_TIME', None), 12.0), (('MOVE_VARIANCE_MULT', None), 0.1), (('PICT_XPOS', ['p1']), 1.0), (('PICT_YPOS', ['p1']), -1.0), (('PICT_VALUE', ['p1']), 5.0), (('PICT_ERROR_ALLOW', ['p1']), 2), (('PICT_XPOS', ['p2']), 1.0), (('PICT_YPOS', ['p2']), 1.0), (('PICT_VALUE', ['p2']), 10.0), (('PICT_ERROR_ALLOW', ['p2']), 0.2), (('PICT_XPOS', ['p3']), 2.0), (('PICT_YPOS', ['p3']), -1.0), (('PICT_VALUE', ['p3']), 7.0), (('PICT_ERROR_ALLOW', ['p3']), 1.5)]


### State and Intermediate Fluents

Quantities that vary over time are represented with state and intermediate fluents. The former map directly to the notion of _state variables_ as defined by [Frances and Geffner (2015)](http://www.dtic.upf.edu/~hgeffner/guillem-icaps-2015.pdf). The latter has come up in the planning literature in a number of different ways, such as _derived predicates_ (see [Hoffman and Edelkamp (2006)](https://courses.cs.washington.edu/courses/cse473/06sp/pddl2.2.pdf) for a formulation specific to ground atomic predicates) or _correlated effects_ (see [Littmann (1997)](http://www.aaai.org/Papers/AAAI/1997/AAAI97-116.pdf) for a discussion in the context of probabilistic planning).

In [19]:
state_vars = domain.state_fluents

for k,v in state_vars.items():
    params = []
    if v.param_types is not None:
        params = v.param_types
    signature = tuple(params + [v.range])
    print('Symbol: {} Signature: {}'.format(k, signature))

Symbol: xPos/0 Signature: ('real',)
Symbol: picTaken/1 Signature: ('picture-point', 'bool')
Symbol: yPos/0 Signature: ('real',)
Symbol: time/0 Signature: ('real',)


In [20]:
interm_vars = domain.intermediate_fluents

for k,v in interm_vars.items():
    params = []
    if v.param_types is not None:
        params = v.param_types
    signature = tuple(params + [v.range])
    print('Symbol: {} Signature: {}'.format(k, signature))

### Action Fluents

RDDL defines actions differently from most "traditional" planning languages, adopting an approach which resembles the standard approach in control, where variables are divided into two sorts: "states" and "inputs". These input variables can be boolean, indicating that the actual effect is turned on or off, or a real/integer value.

In [21]:
interm_vars = domain.action_fluents

for k,v in interm_vars.items():
    params = []
    if v.param_types is not None:
        params = v.param_types
    signature = tuple(params + [v.range])
    print('Symbol: {} Signature: {}'.format(k, signature))

Symbol: snapPicture/0 Signature: ('bool',)
Symbol: yMove/0 Signature: ('real',)
Symbol: xMove/0 Signature: ('real',)


The variables ```xMove``` and ```yMove``` above represent the velocities of the Rover, which can be set directly in this model. ```snapPicture``` indicates that a picture is to be taken at that time instant. Action preconditions are actually represented as state constraints, like:

$$
    snapPicture \supset (xMove = 0) \land (yMove = 0)
$$

which indicate that ```snapPicture``` whenever the Rover is stationary. These precondidtions are given as expressions, combining logical and arithmetic standard operators with variables and constants.

In [22]:
precs = domain.preconds

for prec in domain.preconds:
    print(prec)

Expression(etype=('boolean', '=>'), args=
    Expression(etype=('pvar', 'snapPicture'), args=('snapPicture', None))
    Expression(etype=('boolean', '^'), args=
        Expression(etype=('relational', '=='), args=
            Expression(etype=('pvar', 'xMove'), args=('xMove', None))
            Expression(etype=('number', "<class 'float'>"), args=0.0))
        Expression(etype=('relational', '=='), args=
            Expression(etype=('pvar', 'yMove'), args=('yMove', None))
            Expression(etype=('number', "<class 'float'>"), args=0.0))))


### Translating Preconditions

In [23]:
import tarski.syntax as ts
import tarski.syntax.arithmetic as tm
import tarski.syntax.arithmetic.special as tmsp

In [24]:
for prec in domain.preconds:
    t_prec = rddl.translate_expression( mr_reader.language, prec)
    print(str(t_prec))

(not (snapPicture()) or (=(xMove(),0.0) and =(yMove(),0.0)))


### Action Constraints

In [25]:
act_constraints = domain.constraints

for C in act_constraints:
    print(C)

### State Constraints

RDDL _state invariants_ correspond to state constraints, that define the set of valid states

In [26]:
invariants = domain.invariants

for I in invariants:
    print(I)

### Conditional Probabilistic Functions

In [27]:
for f in domain.state_cpfs:
    #print(f.pvar)
    lhs = rddl.translate_expression(mr_reader.language, f.pvar)
    #print(lhs)
    cpfs_k = rddl.translate_expression(mr_reader.language, f.expr)
    #print('CPFS: lhs: {} rhs: {}'.format(lhs, cpfs_k))

In [28]:
for f in domain.intermediate_cpfs:
    print(f)

## Initial States

In [30]:
instance.max_nondef_actions

'pos-inf'

In [31]:
instance.discount

1.0

In [32]:
instance.horizon

40

In [33]:
print(instance.init_state)

[(('xPos', None), 0.0), (('yPos', None), 0.0), (('time', None), 0.0)]
